In [7]:
import json
# import pydantic

In [8]:
import pandas as pd

In [9]:
with open("../target/manifest.json") as f:
    data = json.load(f)

In [10]:
with open("../target/catalog.json") as f:
    catalog = json.load(f)

In [11]:
# def get_types(node):
#     columns = node['columns']
#     tup_columns = [(node['unique_id'], col['name'].lower(), col['type']) for col in columns.values()]
#     return tup_columns

from typing import Dict, List, Optional
from enum import Enum
from pathlib import Path

from pydantic import BaseModel, validator


class DbtResourceType(str, Enum):
    model = 'model'
    analysis = 'analysis'
    test = 'test'
    operation = 'operation'
    seed = 'seed'
    source = 'source'


class DbtMaterializationType(str, Enum):
    table = 'table'
    view = 'view'
    incremental = 'incremental'
    ephemeral = 'ephemeral'
    seed = 'seed'
    materialized = 'materialzed'
    test = 'test'

# class Column(BaseModel):
#     name = str
#     description = str
    
# class Columns(BaseModel):
#     columns: List[Column]
    
class NodeDeps(BaseModel):
    nodes: List[str]


class NodeConfig(BaseModel):
    materialized: Optional[DbtMaterializationType]


class Node(BaseModel):
    unique_id: str
    path: Path
    resource_type: DbtResourceType
    description: str
    depends_on: Optional[NodeDeps]
    config: NodeConfig
#     columns: Optional[Columns]


class Manifest(BaseModel):
    nodes: Dict["str", Node]
    sources: Dict["str", Node]

    @validator('nodes', 'sources')
    def filter(cls, val):
        return {k: v for k, v in val.items() if v.resource_type.value in ('model', 'seed', 'source')}


m = Manifest(**data)
# print(m)

m.nodes

m.nodes['model.jaffle_shop.orders']

In [12]:
tables = [node for node in data['nodes'] if node[:5] =='model']

In [13]:
def get_columns(table):
    columns = table['columns']
    tup_columns = [(table['relation_name'],table['name'],table['unique_id'], col['name'], col['description']) for col in columns.values()]
    return tup_columns

In [14]:
tests_l = [node for node in list(data['nodes'].keys()) if node[:4]=='test']

In [15]:
def parse_test(test):
    tup = (test['unique_id'],
           test[ "test_metadata"]['name'],
           test[ "test_metadata"]['kwargs']['column_name'],
           test['depends_on']['nodes'][0],
           test['compiled_sql'].strip()
          )
    return tup

In [16]:
def parse_tests(list_tests):
    tests = [parse_test(test_values) for _, test_values in list_tests.items()]
    return tests

In [17]:
ts = {key:value for key, value in data['nodes'].items() if key in tests_l}

In [18]:
tests_df = pd.DataFrame(parse_tests(ts), columns =['test_id', 'test_type', 'column_name', 'node', 'test_sql'])

In [19]:
# # Save the original `to_html` function to call it later
# pd.DataFrame.base_to_html = pd.DataFrame.to_html
# # Call it here in a controlled way
# pd.DataFrame.to_html = (
#     lambda df, *args, **kwargs: 
#         (df.base_to_html(*args, **kwargs)
#            .replace(r"\n", "<br/>"))
# )

In [20]:
column_list = []
for table in tables:
    column_list.extend(get_columns(data['nodes'][table]))

In [21]:
columns_df = pd.DataFrame(column_list,
                          columns = ['relation', 'table_name', 'node', 'column_name','description']
                         )

In [22]:
columns_df

,relation,table_name,node,column_name,description
0,DBT_TABLEAU.DAVID.customers,customers,model.jaffle_shop.customers,customer_id,This is a unique identifier for a customer
1,DBT_TABLEAU.DAVID.customers,customers,model.jaffle_shop.customers,first_name,Customer's first name. PII.
2,DBT_TABLEAU.DAVID.customers,customers,model.jaffle_shop.customers,last_name,Customer's last name. PII.
3,DBT_TABLEAU.DAVID.customers,customers,model.jaffle_shop.customers,first_order,Date (UTC) of a customer's first order
4,DBT_TABLEAU.DAVID.customers,customers,model.jaffle_shop.customers,most_recent_order,Date (UTC) of a customer's most recent order
5,DBT_TABLEAU.DAVID.customers,customers,model.jaffle_shop.customers,number_of_orders,Count of the number of orders a customer has p...
6,DBT_TABLEAU.DAVID.customers,customers,model.jaffle_shop.customers,customer_lifetime_value,Total value (AUD) of a customer's orders
7,DBT_TABLEAU.DAVID.orders,orders,model.jaffle_shop.orders,order_id,This is a unique identifier for an order
8,DBT_TABLEAU.DAVID.orders,orders,model.jaffle_shop.orders,customer_id,Foreign key to the customers table
9,DBT_TABLEAU.DAVID.orders,orders,model.jaffle_shop.orders,order_date,Date (UTC) that the order was placed


In [23]:
comb = columns_df.merge(tests_df, how='left',
                 left_on = ['node','column_name'],
                 right_on = ['node','column_name'],
                ).fillna('')

In [24]:
comb.head(5)

,relation,table_name,node,column_name,description,test_id,test_type,test_sql
0,DBT_TABLEAU.DAVID.customers,customers,model.jaffle_shop.customers,customer_id,This is a unique identifier for a customer,test.jaffle_shop.unique_customers_customer_id....,unique,"select\n customer_id as unique_field,\n ..."
1,DBT_TABLEAU.DAVID.customers,customers,model.jaffle_shop.customers,customer_id,This is a unique identifier for a customer,test.jaffle_shop.not_null_customers_customer_i...,not_null,select *\nfrom DBT_TABLEAU.DAVID.customers\nwh...
2,DBT_TABLEAU.DAVID.customers,customers,model.jaffle_shop.customers,customer_id,This is a unique identifier for a customer,test.jaffle_shop.relationships_orders_customer...,relationships,with child as (\n select customer_id as fro...
3,DBT_TABLEAU.DAVID.customers,customers,model.jaffle_shop.customers,first_name,Customer's first name. PII.,,,
4,DBT_TABLEAU.DAVID.customers,customers,model.jaffle_shop.customers,last_name,Customer's last name. PII.,,,


In [25]:
catalog['nodes']['model.jaffle_shop.customers']['columns']['customer_id'.upper()]['type']

'NUMBER'

In [26]:
comb['column_type'] = comb.apply(lambda x:
                        catalog['nodes'][x['node']]['columns'][x['column_name'].upper()]['type'],axis =1
                                
                                )

In [27]:
comb.head(5)

,relation,table_name,node,column_name,description,test_id,test_type,test_sql,column_type
0,DBT_TABLEAU.DAVID.customers,customers,model.jaffle_shop.customers,customer_id,This is a unique identifier for a customer,test.jaffle_shop.unique_customers_customer_id....,unique,"select\n customer_id as unique_field,\n ...",NUMBER
1,DBT_TABLEAU.DAVID.customers,customers,model.jaffle_shop.customers,customer_id,This is a unique identifier for a customer,test.jaffle_shop.not_null_customers_customer_i...,not_null,select *\nfrom DBT_TABLEAU.DAVID.customers\nwh...,NUMBER
2,DBT_TABLEAU.DAVID.customers,customers,model.jaffle_shop.customers,customer_id,This is a unique identifier for a customer,test.jaffle_shop.relationships_orders_customer...,relationships,with child as (\n select customer_id as fro...,NUMBER
3,DBT_TABLEAU.DAVID.customers,customers,model.jaffle_shop.customers,first_name,Customer's first name. PII.,,,,TEXT
4,DBT_TABLEAU.DAVID.customers,customers,model.jaffle_shop.customers,last_name,Customer's last name. PII.,,,,TEXT


In [28]:
export = comb.groupby(by = ['relation', 'table_name', 'node', 'column_name', 'column_type','description'],
            as_index=False).agg({'test_type':list, 'test_sql':list})

In [29]:
export['test_type'] = export['test_type'].apply(';'.join )
export['test_sql'] = export['test_sql'].apply(';\n'.join)

In [30]:
# export.to_excel("output.xlsx")

In [31]:
def calc_length(column):
    return max(column.apply(lambda x: max([len(y) for y in x.split('\n')])))

def get_column(number):
    return chr(ord('@')+number)

def get_full_column(number):
    return get_column(number) + ':' + get_column(number)

In [32]:
import xlsxwriter


In [49]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
export.to_excel(writer, sheet_name='Sheet1',
                startrow=1, header=False, index=False)

# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

In [50]:
center = workbook.add_format({
#     'align':    'center',
    'valign':   'vcenter'})



In [51]:
# Set column_widths
for i, col in enumerate(export.columns):
    worksheet.set_column(get_full_column(i+1), min(
        max(calc_length(export[col]), len(col)),
        50), center)

In [52]:
wrap = workbook.add_format({'text_wrap': True})
# Set the format but not the column width.
worksheet.set_column('G:G', calc_length(export['test_sql']), wrap)

0

In [53]:
column_settings = [{'header': column} for column in export.columns]
(max_row, max_col) = export.shape

worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})
worksheet.set_default_row(20)

In [54]:
workbook.close()
